In [ ]:
# DOCUMENT LINK
# https://free-apis.github.io/#/browse
# https://dog.ceo/dog-api/documentation/breed
# https://alexwohlbruck.github.io/cat-facts/

In [1]:
import os
import requests
import pandas as pd 
import duckdb
import numpy as np

In [6]:
def get_data_api():

    empty_list = []

    for i in range(10):
        r1 = requests.get('https://dog.ceo/api/breeds/image/random/50').json()['message']
        for r in r1:
            empty_list.append(r)

    new_list = set(empty_list)
    converted_list = list(new_list)
    return converted_list


In [7]:
def transform(converted_list):
    df = pd.DataFrame(converted_list)
    df.columns = ['images']
    df['dogs_name']  = df['images'].apply(lambda x : x.split('/')[4])
    df_concatenated = df.groupby('dogs_name')['images'].agg(list).reset_index()
    return df_concatenated

In [8]:
def write_data(data):
    conn = duckdb.connect('animal_database.duckdb')
    conn.execute('create table if not exists animals AS SELECT * from dog_table')
    tables = conn.execute("SELECT distinct table_name FROM information_schema.tables WHERE table_schema = 'main';").fetch_df()
    result = conn.execute('SELECT * FROM animals').df()
    conn.close()
    return result, tables

    

In [9]:
# EXTRACT 
converted_list =  get_data_api()


In [10]:
# TRANSFORM
dog_table =  transform(converted_list)

In [11]:
# LOAD
data, list_table = write_data(data=dog_table)

In [12]:
list_table

,table_name
0,animals


In [13]:
data.head()

,dogs_name,images
0,affenpinscher,[https://images.dog.ceo/breeds/affenpinscher/n...
1,african,[https://images.dog.ceo/breeds/african/n021167...
2,airedale,[https://images.dog.ceo/breeds/airedale/n02096...
3,akita,[https://images.dog.ceo/breeds/akita/Akita_hik...
4,appenzeller,[https://images.dog.ceo/breeds/appenzeller/n02...


In [14]:
#output result to file 
data.to_csv('output.csv')